In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
import pytorch
import botorch
import gpytorch
import fastai
from fastai.tabular import *

allData = pd.read_csv('combinedData.csv')
allData = allData.sort_values(by='Polymer Concentration Real')
allData = allData[allData['Fit'] == True]

heatData = allData[allData['Heating'] == True]
noheatData = allData[allData['Heating'] == False]

xHeat = heatData['Polymer Concentration Real']
yHeat = heatData['Elastic Modulus']
xNoHeat = noheatData['Polymer Concentration Real']
yNoHeat = noheatData['Elastic Modulus']

plt.scatter(xHeat,yHeat)
plt.xlabel('Polymer Concentration')
plt.ylabel('Elastic Modulus (bar)')
plt.title('Heating Data')
plt.show()

plt.scatter(xNoHeat,yNoHeat)
plt.xlabel('Polymer Concentration')
plt.ylabel('Elastic Modulus (bar)')
plt.title('No Heating Data')
plt.show()

ModuleNotFoundError: No module named 'pytorch'

UsageError: %%python is a cell magic, but the cell body is empty.


In [ ]:
def plot_percentiles(xs, ys):

    probs = [10, 20, 30, 40, 50, 60, 70, 80, 90]

    percentiles = [np.percentile(ys, prob, axis=0) for prob in probs]

    light="#DCBCBC"
    light_highlight="#C79999"
    mid="#B97C7C"
    mid_highlight="#A25050"
    dark="#8F2727"
    dark_highlight="#7C0000"
    green="#00FF00"

    plt.fill_between(xs, percentiles[0], percentiles[8],
                  facecolor=light, color=light)
    plt.fill_between(xs, percentiles[1], percentiles[7],
                  facecolor=light_highlight, color=light_highlight)
    plt.fill_between(xs, percentiles[2], percentiles[6],
                      facecolor=mid, color=mid)
    plt.fill_between(xs, percentiles[3], percentiles[5],
                      facecolor=mid_highlight, color=mid_highlight)
    plt.plot(xs, percentiles[4], color=dark)

In [ ]:
def MyHeteroskedasticGP(X_train, y_train):

    model = SingleTaskGP(train_X=X_train, train_Y=y_train)
    model.likelihood.noise_covar.register_constraint("raw_noise", GreaterThan(1e-5))

    mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)

    mll.train()
    botorch.optim.fit.fit_gpytorch_scipy(mll)

    mll.eval()
    # test on the training points
    # call if X_test just for ease of use
    X_test = X_train.clone()

    mll.eval()
    with torch.no_grad():
        posterior = mll.model.posterior(X_test)
        test_pred = mll.likelihood(mll.model(X_test))

    with torch.no_grad():
        # watch broadcasting here
        observed_var = torch.tensor(
                           np.power(mll.model.posterior(X_train).mean.numpy().reshape(-1,) - y_train.numpy(), 2),
                           dtype=torch.float
        )

    # NOW TRAIN HETERO MODEL
    model = HeteroskedasticSingleTaskGP(train_X=X_train, train_Y=y_train,
                                    train_Yvar=observed_var)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)

    mll.train()
    botorch.optim.fit.fit_gpytorch_scipy(mll)

    return mll

In [ ]:
X_train = torch.tensor(xHeat.reshape(-1,1), dtype=torch.float)
y_train = torch.tensor(yHeat, dtype=torch.float)

mll = MyHeteroskedasticGP(X_train, y_train)

X_test=X_train.squeeze(-1)
mll.eval()
with torch.no_grad():
    posterior = mll.model.posterior(X_test)

    predictive_noise = torch.exp(mll.model.likelihood.noise_covar.noise_model.posterior(X_test).mean)
    # get standard deviation
    predictive_noise_std = torch.sqrt(predictive_noise).squeeze(-1)

with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(6, 4))

    # fit posterior confidence
    lower, upper = posterior.mvn.confidence_region()
    # get posterior predictive confidence by adding noise from noise model
    lower_predictive = lower - 2*predictive_noise_std
    upper_predictive = upper + 2*predictive_noise_std

    # Plot training data
    ax.scatter(X_train.numpy(), y_train.numpy(), c='b', s=1)
    # Plot predictive means as blue line
    ax.plot(X_test, posterior.mean.numpy(), 'k-')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(X_test.numpy(),
                    lower.numpy(),
                    upper.numpy(), alpha=0.5)
    ax.fill_between(X_test.numpy(),
                    lower_predictive.numpy(),
                    upper_predictive.numpy(),
                    alpha=0.3)
    ax.legend(['Mean', 'Observed Data', 'Posterior Confidence',
               'Posterior Predictive Confidence'])
plt.title('Heteroskedastic GP')